# gathering metadata on assistive technology

using github graphql api to gather metadata about
projects tagged with assistive technology.

this exploration looks specifically at assistive technology tags.
this might be a good _training_ set for a more open general search
that tries to differentiate between assistive tech presenting and the real deal.

In [1]:
    import os, pandas
    from toolz.curried import *
    from pandas import DataFrame, Series, Index
    with __import__("importnb").Notebook():
        from app import App    

The graph QL query is constructed And tuned using the interactive explorer at URL.
Makes sense to have more queries eventually

In [2]:
search_blob = """{
      search(query: "%s", type: REPOSITORY, first: 50, after: %s) {
        repositoryCount
    wikiCount
    pageInfo {
      hasNextPage
      endCursor
    }
    edges {
      node {
        ... on Repository {
          url
          id
          stargazerCount
          forkCount
          description
          issues {
            totalCount
          }
          languages(first: 20) {
            nodes {
              name
            }
          }
          licenseInfo {
            name
          }
          fundingLinks {
            platform
            url
          }      
          pullRequests {
            totalCount
          }
          updatedAt
          object(expression: "HEAD:README.md") {
            ... on Blob {
              text
            }
          }
          repositoryTopics(first: 20) {
            edges {
              node {
                topic {
                  name
                }
              }
            }
          }
        }
      }
    }
      }
}"""

In [3]:
    def get_search_cache(app):
        return app.cache / "github" / "search_responses.pkl"
    @App.impl    
    def set_params(params):
        params.update(Authorization=F"token {os.environ['GITHUB_TOKEN']}")
        
    def get_graphql_client(app):
        from python_graphql_client import GraphqlClient
        return GraphqlClient(
            "https://api.github.com/graphql", app.params
        )

In [4]:
    async def search_one(query, app, options, after=None):
        get_search_cache(app).parent.mkdir(exist_ok=True, parents=True)
        client = get_graphql_client(app)
        with __import__("shelve").open(get_search_cache(app)) as cache:
            id = F"{query}-{after}"
            cache[id] = await client.execute_async(
                search_blob % (query, str(after and F'"{after}"' or "null")))
            return cache[id]


In [5]:
    async def search(query, app, options, stop=20):       
        queries = [await search_one(query, app, options)]
        ct = 0
        print(queries[-1])
        while queries[-1]["data"]["search"]:
            if info := queries[-1]["data"]["search"].get("pageInfo"):
                if info["hasNextPage"]:
                    if app.verbosity:
                        print(queries[-1]["data"]["search"]["pageInfo"]["endCursor"])
                    queries.append(
                        await search_one(query, app, options, queries[-1]["data"]["search"]["pageInfo"]["endCursor"])
                    )
                    if app.verbosity:
                        print(F"done {ct}")
                    ct += 1
                    if ct == stop: break
                    continue
            break
        return queries

In [6]:
@App.impl
async def get_responses(options, app):
    from asyncio import gather
    return await gather(
        search("topic:assistive-technology", app, options, os.environ.get("PAGES", 1)),
        search("topic:screen-reader", app, options, os.environ.get("PAGES", 1))
    )

In [7]:
@App.impl
def get_frames(app):
    with __import__("shelve").open(get_search_cache(app)) as db:
        data = DataFrame(db.values(), db)["data"].apply(Series
        )["search"].apply(Series
        )["edges"].explode(
        ).apply(Series)["node"].apply(Series).set_index("id")
    data.pullRequests = data.pullRequests.apply(operator.itemgetter("totalCount"))
    data.issues = data.issues.apply(operator.itemgetter("totalCount"))
    data["README"] = data.pop("object").apply(lambda x: x and operator.itemgetter("text")(x) or "")

    data = data.join(
        data.pop("repositoryTopics").apply(Series)["edges"]
        .explode().apply(operator.itemgetter("node")).apply(operator.itemgetter("topic")).apply(operator.itemgetter("name"))
        .groupby(level=0).agg(list).rename("topics")
    )
    return data

In [8]:
if __name__ == "__main__":
    app = App()
    app.main(__import__(__name__), run=0 or locals().get("__file__") is not None)
    df = app.compact()
    display(
        df.T, 
        df.topics.explode().value_counts().to_frame("count").T.style.set_caption(
            "the column titles are the tags of the returned search results and their counts are in the cells below"
        )
    )
        

id,R_kgDOItjjpA,MDEwOlJlcG9zaXRvcnkyMTgzNjE0OA==,MDEwOlJlcG9zaXRvcnkzNTM1NjE3OA==,MDEwOlJlcG9zaXRvcnkyMDAyNzUxMjQ=,MDEwOlJlcG9zaXRvcnk4MTU3NTc1OQ==,R_kgDOJb8f4A,R_kgDOKrDHjA,R_kgDOGox2Sw,R_kgDOJ4lpzw,MDEwOlJlcG9zaXRvcnkzNDAwMzA2MzA=,...,MDEwOlJlcG9zaXRvcnkzMjg2MjUwMjg=,R_kgDONLjnqw,R_kgDONBSJWg,R_kgDOOWXE9Q,R_kgDOIzKasg,MDEwOlJlcG9zaXRvcnkxMDEyMTc2MTQ=,MDEwOlJlcG9zaXRvcnkxOTUxNjA5MA==,R_kgDOONatUQ,R_kgDOHsjedw,R_kgDOHQTMEg
url,https://github.com/ai-collection/ai-collection,https://github.com/OptiKey/OptiKey,https://github.com/brunopulis/awesome-a11y,https://github.com/Stypox/dicio-android,https://github.com/cboard-org/cboard,https://github.com/themanyone/whisper_dictation,https://github.com/voqal/voqal,https://github.com/HARPLab/DReyeVR,https://github.com/tmc/righthand,https://github.com/pedrooaugusto/speech-to-code,...,https://github.com/Agathebadia/inclusive-desig...,https://github.com/ScaerieTale/S3R-Screen-Reader,https://github.com/mafSida/green-harvest-team-...,https://github.com/emirbaycan/ocr_digital_scre...,https://github.com/sig-a11y/BaoYi-fake-nvdaCon...,https://github.com/hatemile/firefox-hatemile-f...,https://github.com/hatemile/hatemile-for-php,https://github.com/code-with-amirhossein/a11y,https://github.com/sophiamersmann/friendly-charts,https://github.com/arunpraba/useAccessibleTime...
stargazerCount,8280,4344,1896,1011,696,254,186,176,113,86,...,0,0,0,0,0,0,0,0,0,0
forkCount,827,504,144,100,215,32,12,50,10,12,...,0,0,4,0,1,1,1,0,0,0
description,The Generative AI Landscape - A Collection of ...,OptiKey - Full computer control and speech wit...,A curate list about A11Y,Dicio assistant app for Android,Augmentative and Alternative Communication (AA...,"Private voice keyboard, AI chat, images, webca...",Voice native AI agent for the builders of tomo...,VR driving 🚙 + eye tracking 👀 simulator based ...,RightHand - A GPT4 powered assistive tool.,Speech to Code - Enables you to code using jus...,...,"Implement & Inclusive design, components, for ...",Super Simple Screen Reader lets you paste any ...,"Team project built with HTML, CSS, JS, and Vit...",OCR tool for extracting numbers from digital s...,"A fake nvdaControllerClient.dll, but take BaoY...","Improve the accessibility of web pages, especi...",HaTeMiLe is a library that can convert a HTML ...,A short look on accessibility in front-end dev...,Screen reader friendly charts with minimal effort,useAccessibleScreenReaderTimeCounter is a hook...
issues,59,516,20,208,671,9,5,146,2,3,...,0,0,0,0,0,1,0,0,0,0
languages,{'nodes': []},"{'nodes': [{'name': 'C#'}, {'name': 'HTML'}, {...",{'nodes': []},"{'nodes': [{'name': 'Ruby'}, {'name': 'Shell'}...","{'nodes': [{'name': 'HTML'}, {'name': 'CSS'}, ...",{'nodes': [{'name': 'Python'}]},"{'nodes': [{'name': 'JavaScript'}, {'name': 'K...","{'nodes': [{'name': 'C++'}, {'name': 'C#'}, {'...",{'nodes': [{'name': 'Go'}]},"{'nodes': [{'name': 'TypeScript'}, {'name': 'P...",...,"{'nodes': [{'name': 'CSS'}, {'name': 'HTML'}]}","{'nodes': [{'name': 'CSS'}, {'name': 'HTML'}, ...","{'nodes': [{'name': 'CSS'}, {'name': 'HTML'}, ...",{'nodes': [{'name': 'Python'}]},"{'nodes': [{'name': 'C'}, {'name': 'C++'}]}","{'nodes': [{'name': 'JavaScript'}, {'name': 'C...","{'nodes': [{'name': 'PHP'}, {'name': 'JavaScri...","{'nodes': [{'name': 'TypeScript'}, {'name': 'C...",{'nodes': [{'name': 'TypeScript'}]},"{'nodes': [{'name': 'HTML'}, {'name': 'JavaScr..."
licenseInfo,{'name': 'MIT License'},{'name': 'GNU General Public License v3.0'},{'name': 'Creative Commons Zero v1.0 Universal'},{'name': 'GNU General Public License v3.0'},{'name': 'GNU General Public License v3.0'},{'name': 'GNU General Public License v2.0'},{'name': 'Apache License 2.0'},{'name': 'MIT License'},None,{'name': 'GNU General Public License v3.0'},...,None,None,None,{'name': 'GNU General Public License v3.0'},None,{'name': 'Apache License 2.0'},{'name': 'Apache License 2.0'},None,{'name': 'MIT License'},None
fundingLinks,"[{'platform': 'LIBERAPAY', 'url': 'https://lib...","[{'pla

topics,assistive-technology,screen-reader,accessibility,a11y,wcag,wai-aria,nvda,python,text-to-speech,ai,openat,aac,aria,machine-learning,a11y-testing,assistant,voiceover,a11y-automation,voice-assistant,javascript,accessibility-testing,screen-reader-api,accessibility-automation,python3,mouse-emulation,windows,react,tts,open-source,chrome-extension,blind,artificial-intelligence,eye-tracking,joystick,web-accessibility,inclusive-design,a11y-api,nvda-addon,ocr,arduino,android,firefox-extension,asterics-main,computer-vision,human-computer-interaction,robotics,disability,deep-learning,mouth-mouse,awesome,linux,voice-recognition,screenreader,opencv,haskell,typescript,opera-extension,html,scala,jaws,speech-synthesis,scalameta,hardware,neovim-plugin,lip-mouse,visually-impaired,assistive,keyboard-emulation,vscode-extension,cli,youtube,augmentative-and-alternative-communication,cross-platform,personal-assistant-framework,keyboard,documentation,mouse,personal-assistant,gaming,speech-to-code,html5,disabilities,rust,speech-recognition,screenshot,screen-capture,eye-tracker,assistant-chat-bots,voice-programming,switch-control,virtual-keyboard,llm,dicio,awesome-list,dicio-assistant,communication,image-processing,braille,mp3,special-needs,css,java,hacktoberfest,firmware,color-contrast,twitter,natural-language-processing,nodejs,esp32,responsive-design,mit-license,ble,bluetooth,speech-to-text,example,switch,open-hardware,screen-readers,bot,keyboard-navigation,mac,screen-recorder,screenshots,screenshot-utility,epub,narrator,macos,english,angular,automation,ui-automation,playwright,nvdaaddon,sign-language,terminal,jamstack,cpp,ollama,asl,macosx,neural-network,ape,demo,at-solution,biomechanics,kotlin,assistive-robotics,bci,brain-computer-interface,eeg,opensource,user-experience,circuitpython,3d-printing,gpt4,virtual-reality,augmentative-communication,claude,voice-user-interface,pwa,voice-commands,voice-control,android-application,environmental-control,speech,golang,language-model,accessible,electron,vscode,lipsync,pcb,software-development,mediapipe,face-tracking,stl,diy,signal-processing,timestamp,datetime,spanish,a11y-experiment,aria-label,optikey,time,vuejs,date,tetris,vue-a11y,ngx-a11y,screen-keyboard,colaboratory,data-science,pdf-converter,webdriver,automated-tests,tree-of-lens,layout-understanding,gui-agents,grounding,ai-agents,mnd,aria-roles,zdsr,sapi5,eyetracking,universalspeech,screensaver,python37,add-on,audiogame,audiogames,nvdasr,speech-dispatcher,astrojs,astrojs-boilerplate,astrojs-framework,astrojs-template,cms,decap-cms,pass-text,wxpython,wikipedia,wikisearch,python-,mobile,eyes,disabilty,react-component,web,godot,pharo,smalltalk,fast-sapi,voice-over,c-sharp,av-speech,als,tty,genai,switch-access,education,ruby-on-rails,openai,react-bootstrap,reactjs,adapted-gaming,openhardware,visualization,c-plus-plus,star-trek,whisper-cpp,whisper-api,stable-diffusion-webui,screencast,focus-management,drop-down,repl,shitposting,auto-complete,markov,augmented-reality,ios,real-time-translation,trackpoint,cirque,xbox-controller,openscad,xbox,text-to-braille,braille-to-text,braille-education,stable-diffusion,server,launcher,progressive-web-app,data-visualization,vosk,console,python-3,autism,ibmtts,playwright-a11y,cerebral-palsy,minecraft,mod,minecraft-mod,image-matching,screen-scraping,image-recognition,iaccessible2,hands-free,ia2,communication-board,talkback,rsr,msaa,java-access-bridge,jab,symbols,client,client-server,coding,continuous,dictation,desktop-duplication-api,github-pages,generative-testing,generative-text,offline,blind-people,acessibility,announcement,live-regions,service-worker,aria-recommendations,pdf-accessibility,exam-accessibility,workshop-materials,circleci,frontend-mentor,aria-attributes,scss,accesibility,wordpress-plugin,unipd,schema-driven-development,vercel,creative-tools,design-tools,figma,figma-community,next-js,plugin,vision,boyctrl,google-text-to-speech,pakistan,urdu,polymer,mongodb,generative-art,javaee,stock-hawk,stock-quote,da